In [1]:
# define the network

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # backward is not necessary; you can just use autograd

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [2]:
# learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv2's .weight
print(params[3].size())  # fc1's .weight

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16])


In [3]:
# you can feed the net like so:
input = Variable(torch.randn(1, 1, 32, 32)) # just a Variable
out = net(input) # this is also a variable
print(out)

Variable containing:
 0.0713  0.1237  0.0063 -0.0589  0.0032 -0.0180  0.0722  0.0360 -0.0238  0.0178
[torch.FloatTensor of size 1x10]



In [4]:
# zero out the gradients and do back propagation with random gradients.
net.zero_grad()
out.backward(torch.randn(1, 10))

In [5]:
# loss function
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.3497
[torch.FloatTensor of size 1]



In [6]:
# now that we calculated the loss, we can now back propagate to calculate graidents
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.0066
-0.0228
 0.0380
-0.0697
 0.0561
-0.1608
[torch.FloatTensor of size 6]



In [7]:
# now we can update the weights using the gradients
import torch.optim as optim

# create your optimizer (Adam, others like SGD available)
optimizer = optim.Adam(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update